# **Instituto Tecnológico Vale**

## **Introdução a Visão Computacional**

### **Aula 06 - Detecção de objetos com Faster RCNN**

In [ ]:
!wget -O coco_classes.pickle https://www.dropbox.com/s/zfrbfi9o10wplvf/coco_classes.pickle?dl=0

!wget -O transito.jpg https://www.dropbox.com/s/r9hg5mt2fllcoz3/transito.jpg?dl=0

In [ ]:
# import the necessary packages

from torchvision.models import detection
import numpy as np
import argparse
import pickle
import torch
import cv2

from google.colab.patches import cv2_imshow

In [ ]:
# define o dispositivo que usaremos para executar o modelo
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# carregua a lista de categorias no conjunto de dados COCO e, 
# em seguida, gere um conjunto de cores da caixa delimitadora para cada classe

CLASSES = pickle.loads(open("/content/coco_classes.pickle", "rb").read())

COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

In [ ]:
# Defina uma threshold para a confiança da detecção

cf_thresh = 0.7

In [ ]:
# inicializa um dicionário contendo o nome do modelo e sua chamada de função torchvision correspondente

MODELS = {
	"frcnn-resnet": detection.fasterrcnn_resnet50_fpn,
	"frcnn-mobilenet": detection.fasterrcnn_mobilenet_v3_large_320_fpn,
	"retinanet": detection.retinanet_resnet50_fpn
}

In [ ]:
# carregua o modelo e configure-o para o modo de avaliação

# DIgite aqui o nome da rede que você quer usar. Escolha uma das três indicadas em MODELS
key = 'frcnn-resnet'

model = MODELS[key](pretrained=True, progress=True,
                    num_classes=len(CLASSES),
                    pretrained_backbone=True).to(DEVICE)

model.eval()

In [ ]:
# carregua a imagem do disco

# Escolha a sua imagem!

image = cv2.imread('/content/transito.jpg')
orig = image.copy()

In [ ]:
#converter a imagem de BGR para a ordem de canal RGB e 
# mudar a ordem dos canais para o formtao que pytorch aceita [c, w, h]

image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = image.transpose((2, 0, 1))

In [ ]:
# Adiciona a dimensão do lote, 
# dimensiona as intensidades de pixel bruto para o intervalo [0, 1] 
# e converte a imagem em um tensor de ponto flutuante

image = np.expand_dims(image, axis=0)
image = image / 255.0
image = torch.FloatTensor(image)

In [ ]:
# envia a imahgem para o dispositivo da (GPU) e passa pela rede para obter as detecções e previsões

image = image.to(DEVICE)
detections = model(image)[0]

In [ ]:
# loop over the detections
for i in range(0, len(detections["boxes"])):
	# extract the confidence (i.e., probability) associated with the
	# prediction
	confidence = detections["scores"][i]

	# filter out weak detections by ensuring the confidence is
	# greater than the minimum confidence
	if confidence > cf_thresh:
		# extract the index of the class label from the detections,
		# then compute the (x, y)-coordinates of the bounding box
		# for the object
		idx = int(detections["labels"][i])
		box = detections["boxes"][i].detach().cpu().numpy()
		(startX, startY, endX, endY) = box.astype("int")

		# display the prediction to our terminal
		label = "{}: {:.2f}%".format(CLASSES[idx], confidence * 100)
		print("[INFO] {}".format(label))

		# draw the bounding box and label on the image
		cv2.rectangle(orig, (startX, startY), (endX, endY),
			COLORS[idx], 2)
		y = startY - 15 if startY - 15 > 15 else startY + 15
		cv2.putText(orig, label, (startX, y),
			cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)

In [ ]:
# Mostra a imagem com as detecções

cv2_imshow(orig)
cv2.waitKey(0)